(sec-floating-point)=
# Floating Point Numbers

Mathematically, real numbers are continuous and there are uncountablly many of them. There is no way to express real numbers in discrete systems. Therefore, it is quite challenging to express real numbers accurately in computers. A clever method known as _floating point arithmetic_ was developed and it is now a standard method to implement real numbers in modern computers.[^1]  Since scientific computation relies on the properties of floating point, we need to understand them.{cite}`Goldberg1991`

[^1]: The latest standard is published in 2019, which is known as IEEE754-2019 standard.

In the floating point arithmetic, a real number is expressed in 
scientific notation such as $1325.67 \times 10^{12}$.  In computer languages, it is usually expressed as 1325.67E12 or 1325.67e12.  The idea is simple. The mantissa and exponent of scientific notation are treated separately as two integers. For example, 1325.67E12 = 132567E10 can be expressed with two integers 132567 and 10.  The significant figure is determined by the size of integer expressing the mantissa.

Real numbers stored in a 32-bit string is known as type `float32` or _single precision_. It uses 24 bits for mantissa and 8 bits for exponent.  One bit is used to specify $\pm$ sign.  The corresponding significant figure is $\log_{10} 2^{23} \approx 7$.  The exponent part is $2^{2^7}=2^{-128}$ to $2^{2^7-1}=2^{127}$ which is approximately $10^{-38}$ to $10^{+38}$. A strange feature of the standard floating point arithmetic is that there are two different zeroes, $+0.0$ and $-0.0$.  This is not a bug. Inside the computers, they are treated as two different numbers.  

Real numbers stored in a 64-bit string is known as type `float64` (simply `float` in python) or _double precision_. It uses 54 bits for mantissa and 10 bits for exponent as shown in {numref}`fig-double-float`.  The largest value the mantissa can express  is $2^{53} = 9007,199,254,740,992$, which corresponds to significant figure 16.  The maximum exponent part is between $2^{-2^9} = 2^{-512} \approx 10^{-308}$ and $2^{2^9-1} = 2^{511} \approx 10^{308}$.[^2][^3] 



Usually, the single precision (`float32`) is not accurate enough for computational physics and thus we should use the double precision (`float64`).  In python core the double precision (`python float`) is default. The `math` package uses the python float. The double precision (`float64`) is default in `numpy`.[^4][^5]  Unlike integer type, python float and `numpy` float are the same and thus we don;t worry about conversion from python to `numpy`.

[^2]: There is a slightly better enconding known as *denormalized float*.  The smallest value in the denormalized float method is $4.9406564584124654 \times 10^{-324}$ for double and $1.401298 \times 10^{-45}$ for single  which is smaller than the smallest value in the standard floating point method.  Most of modern computer programming language use denormalized float for very small number.  However, if the smallest possible value is asked, the system still returns the value in the standard floating point. 

[^3]: Modern CPUs used in desktop and laptop computers are capable of 80 bits floating point arithmetic which is more accurate than `float64`.  C/C++ offeres `long double` type for it.  Similarly, Fortran allows `REAL*10` which utilizes the 80 bits floating point processor.  However, they are not a standard size in the current computational environment.

Some advanced computers are equipped with special arithmetic engine capable of 128-bits floating point arithmetic. The default size of floating point in python is 64 and its type is just `float`, which is good enough for most of numerical calculation in physics.  In `numpy`, we can use both `float32` and `float64` (default is `float64`). [^2]

[^4]:`numpy` also offers `float128` but it is not true `float128` and not reliable at the present.

[^5]: Some advanced computers are equipped with a special arithmetic engine capable of 128-bits floating point arithmetic. 

```{figure} double-float.png
---
name: fig-double-float
---
64-bit string for floating point expression.  The last bit is used for the sign and 11 bits from $b_{52}$ to $b_{62}$ express the exponent.  The remaining 52 bits express the mantissa.
```

## The range of floating points

As discussed above, floating point has a finite range based on the size of bit string.  In most computers, the range is

|  Type  | Minimum value   |  Maximum value |
| ------ | -------------   | -------------- |
| single | 1.175494351E-38 | 3.402823466E+38|
| double | 2.2250738585072014E-308 | 1.7976931348623158E+308|

You don't have to memorize these numbers since pythonknows them.  The follwoing example extract those information from python. 



---

**Example 1.4.1**: Range of floating point numbers

Let us try to find the largest and smallest *positive* numbers in your computer system.  `float_info` class in `sys` package tells the range of python float and the same information in `numpy` can be obtained with `np.finfo.  The both should output the same value.

In [20]:
# float in python core
import sys

python_fmin = sys.float_info.min
python_fmax = sys.float_info.max

# float in numpy
import numpy as np

numpy_fmin = np.finfo(np.float64).tiny
numpy_fmax = np.finfo(np.float64).max

print("python float type =",type(python_fmax))
print(" numpy float type =",type(numpy_fmax))
print()
print("python smallest positive float =",python_fmin)
print(" numpy smallest positive float =",numpy_fmin)
print()
print("python largest float =",python_fmax)
print(" numpy largest float =",numpy_fmax)


python float type = <class 'float'>
 numpy float type = <class 'numpy.float64'>

python smallest positive float = 2.2250738585072014e-308
 numpy smallest positive float = 2.2250738585072014e-308

python largest float = 1.7976931348623157e+308
 numpy largest float = 1.7976931348623157e+308


## Special value "inf"

If the value exceeds the max, python outputs "inf".  Although it is not real infinity, python thinks it is.  Getting inf means your calculation failed due to *overflow error*.


---
**Example 1.4.2**: number above fmax

Find what is the outcome of a number larger than the maximum or smaller than minimum.

In [2]:
print("2 x fmax =", 2*fmax)

2 x fmax = inf


## Special value "nan"

If mathematical operation is undefined, python just outputs "nan" whcih stands for "not a number".



---
**Example 1.4.3**: Infinity - Infinity

$\infty - \infty$ is not zero.  The result is undefined.  If the result of a mathematical operation is undefined, python declares it is not a number or `nan`.  Let us computer $2 \times \infty - \infty$.  The floating point expression of $\infty$ is given by `float('inf')` which is _a number_ `inf`.  However, `float('inf')*2 - float('inf')` is `nan`, which is _not a number_.

In [25]:
x = float('inf')
print('x =',x)
print('x*2-x =',x*2-x)

x = inf
x*2-x = nan


## Overflow errors

When the output of a calculation exceeds the maximum of floating point (fmax), normally you get `inf`. In som cases, python issues `overflow` error instead.  Usually, overflow error happens when you enter too large value into a function.  For example `exp(1000)` causes overflow error.  `inf` is just a _number_ and not an error, the computation is not interupted. However, overflow error is serious and the compuation stops there.  There is no universal mitigation of overflow error.  You need to find a better algorithm to compute it. 
The following example explains how to evaluate factorial of a large integer.


---
**Example 1.4.4**: Factorial

Factorial of a large integer is astronomically large. For example, 1000!.  Let try to compute 100! first.  While the paython core can deal with arbitrary large integer numbers. It does not provide mathematical functions.  For mathematical computation, ```numpy``` package is a common choice but it can't deal with a huge integer beyond we discussed in Section {numref}`sec-integers`.  Now we resort to ```math``` package which can handle arbitrary large integer number in mathetical computation. 

In [19]:
# Here we use math package to compute a large factorial.
import math

# 100! using math factorial function

x=math.factorial(100)
print('100! =',x)


100! = 93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000


It is too large to store in 64-bit string discussed in Section {numref}`sec-integers`. Our brain cannot comprehend such value.  It is better to express it approximaltely in scientific notation.  In phython, that is floating point expression.


In [14]:
# converting x=100! to float

print('Approxiamate value in float =',float(x))

Approxiamate value in float = 9.332621544394415e+157


So, $100! \approx 9.33 \times 10^{157}$. It fits to 64bit floating point expression.  Now, try 1000!

In [15]:
x=math.factorial(1000)
print('1000! =',x)

1000! = 40238726007709377354370243392300398571937486421071463254379991042993851239862902059204420848696940480047998861019719605863166687299480855890132382966994459099742450408707375991882362772718873251977950595099527612087497546249704360141827809464649629105639388743788648733711918104582578364784997701247663288983595573543251318532395846307555740911426241747434934755342864657661166779739666882029120737914385371958824980812686783837455973174613608537953452422158659320192809087829730843139284440328123155861103697680135730421616874760967587134831202547858932076716913244842623613141250878020800026168315102734182797770478463586817016436502415369139828126481021309276124489635992870511496497541990934222156683257208082133318611681155361583654698404670897560290095053761647584772842188967964624494516076535340819890138544248798495995331910172335555660213945039973628075013783761530712776192684903435262520001588853514733161170210396817592151090778801939317811419454525722386554146106289218796022383

which is hugh and practically useless.  Let us convert it to float.

In [19]:
print('Approxiamate value in float =',float(x))

NameError: name 'math' is not defined

The direct evaluation of 1000! seems hopeless.  We need to find a manual way to find its scientific notation
 $1000! \approx a \times 10^{b}$.  In order to find the mantissa $a$ and exponent $b$, first we evaluate $\log N!$ as follows.

$$
\begin{eqnarray}
y &=& \log(N!) = \log(1 \cdot 2 \cdot 3 \cdots N-1 \cdot N) \\
&=& \log(1)+\log(2)+\log(3)+\cdots + \log(N-1)+\log(N)
\end{eqnarray}
$$

where we assume the base is 10, that is  $\log = \log_{10}$
Notice that $\log 1000 = 3$.  Hence, $\log N!$ is just a sum of small number.  Once we found $y$, the factorial can be obtained by $n! = 10^y$. 
Next we split $y$ to the whole number k=$\lfloor y \rfloor$ and the residual $\delta=y - \lfloor y \rfloor$.
Now, we have $n! = 10^{k+\delta} = 10^\delta \times 10^k$ and thus the mantissa is $10^\delta$ and power is $k$.  

In [27]:
import math
x = math.factorial(1000)
y = math.log10(x)
b = math.floor(y)
a = 10**(y-b)

print("power=",b)
print("mantissa=",a)

power= 2567
mantissa= 4.023872600773956


which tells that $1000! \approx 4.0239 \times 10^{2567}$.  You can see that the number is far bigger the maximum of `float64`.


---
Last modified on 02/09/2024 by R. Kawai.